In [ ]:
import json
import requests
import bibtexparser

In [ ]:
def fetch_metadata_uid_using_doi(doi, api_key):
    url = f"https://api.clarivate.com/api/wos?databaseId=WOK&usrQuery=DO=({doi})"
    headers = {
        'X-ApiKey': api_key
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f'\nReceiving response for DOI {doi}:\n{response.json()}')
        uid = ''
        if response.json()['QueryResult']['RecordsFound'] > 0:
            uid = response.json()['Data']['Records']['records']['REC'][0]['UID']
            print('UID : ', uid.replace(':', '%3A'))
        return response.json(), uid.replace(':', '%3A')
    else:
        print(f"\nError fetching metadata for DOI {doi}: {response.status_code}")
        print("Response Content:", response.content)
        return None, None

In [ ]:
def fetch_references_using_uid(uid, api_key):
    all_references = {'Data':[], 'QueryResult':{}}
    first_index = 1
    count = 100

    while True:
        url = f"https://api.clarivate.com/api/wos/references?databaseId=WOK&uniqueId={uid}&count={count}&firstRecord={first_index}"
        headers = {
            'X-ApiKey': api_key
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            references = response.json()
            all_references['Data'].extend(references['Data'])
            all_references['QueryResult'] = references['QueryResult']
            records_found = references.get('QueryResult', {}).get('RecordsFound', 0)
            if first_index + count > records_found:
                break
            first_index += count
        else:
            print(f"\nError fetching references for UID {uid}: {response.status_code}")
            print("Response Content:", response.content)
            break
    print(f'\nReceiving references for UID {uid}:\n{all_references}\n')
    return all_references

In [ ]:
def extract_dois(references):
    dois = []
    for ref in references['Data']:
        doi = ref.get('DOI')
        if doi:
            doi = doi.replace('/', '%2F').replace('.', '%2E').replace('-', '%2D')
            dois.append(doi)
    return dois

In [ ]:
def process_papers(papers, api_key, depth, visited_dois, current_depth=0): ######
    if current_depth > depth:
        return []

    processed_papers = []

    p_counter = 0

    for paper in papers:
        p_counter += 1
        print('Counter : ', p_counter, '/', len(papers))
        if isinstance(paper, dict):
            paper_doi = paper.get('doi')
        else:
            paper_doi = paper
        
        if paper_doi and paper_doi not in visited_dois:  ######
            visited_dois.add(paper_doi) ######
            metadata, uid = fetch_metadata_uid_using_doi(paper_doi, api_key)
            if uid:
                references_data = fetch_references_using_uid(uid, api_key)
                if references_data:
                    reference_dois = extract_dois(references_data)
                    references = process_papers(reference_dois, api_key, depth, visited_dois, current_depth + 1) ######
                    processed_papers.append({
                        'doi': paper_doi,
                        'metadata': metadata,
                        'references': references
                    })

    return processed_papers

In [ ]:
def main():

    with open('seedPapers.bib') as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    
    papers = bib_database.entries

    api_key = 'your-api-key'
    depth = 1
    visited_dois = set() ######

    all_papers_data = process_papers(papers, api_key, depth, visited_dois) ######

    with open('all_papers_data.json', 'w') as f:
        json.dump(all_papers_data, f, indent=4)
        

if __name__ == "__main__":
    main()